## Masking

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
obs_file='/home/pdirmeye/classes/clim680_2022/OISSTv2/monthly/sst.mnmean.nc'
ds_obs=xr.open_dataset(obs_file)
ds_obs

In [ ]:
plt.contourf(ds_obs.lon,ds_obs.lat,ds_obs['sst'].mean('time'),cmap='coolwarm')
plt.colorbar()

In [ ]:
mask_file='/home/pdirmeye/classes/clim680_2022/OISSTv2/lmask/lsmask.nc'
ds_mask=xr.open_dataset(mask_file)
ds_mask

In [ ]:
plt.contourf(ds_mask['mask'])

In [ ]:
ds_mask=ds_mask.squeeze()
plt.pcolormesh(ds_mask['mask'])
plt.colorbar()

In [ ]:
plt.pcolormesh(ds_mask.lon,ds_mask.lat,ds_mask['mask'])
plt.colorbar()

In [ ]:
da_ocean=ds_obs['sst'].mean('time').where(ds_mask['mask']==1)

# Set the axes using the specified map projection
ax=plt.axes(projection=ccrs.PlateCarree())

# Make a filled contour plot
cs = ax.pcolormesh(da_ocean.lon,da_ocean.lat,da_ocean,cmap='coolwarm',
                 transform = ccrs.PlateCarree())
cbar = plt.colorbar(cs,orientation='horizontal',label='Mean SST (˚C)') 

# Add coastlines
ax.coastlines() 

In [ ]:
weights = np.cos(np.deg2rad(ds_mask.lat))
unmasked = ds_obs['sst'].weighted(weights).mean(dim=['lon','lat'])
masked = ds_obs['sst'].where(ds_mask['mask']==1).weighted(weights).mean(dim=['lon','lat'])

In [ ]:
plt.plot(masked.time,masked,label='Land masked out')
plt.plot(unmasked.time,unmasked,label='No mask applied')
plt.title('Global mean OI SST')
plt.ylabel('Degrees Celsius')
plt.legend()

## Interpolation

In [ ]:
model_path = '/home/pdirmeye/classes/clim680_2022/CCCma.CanCM4/'
model_file = 'ts_Amon_CanCM4_historical_r1i1p1_196101-200512.nc'
ds_model = xr.open_dataset(model_path+model_file)
ds_model

In [ ]:
ds_model['ts'] = ds_model['ts'] - 273.15
ds_model['ts'].attrs['units'] = ds_obs['sst'].attrs['units']
ds_model['ts']

In [ ]:
ds_obs_mean = ds_obs.mean(dim='time')
ds_model_mean = ds_model.mean(dim='time')
ds_model_mean

In [ ]:
ds_model_mean = ds_model_mean.rename({'ts':'sst'})

In [ ]:
model_interp = ds_model_mean.interp_like(ds_obs_mean)
model_interp

In [ ]:
fig = plt.figure(figsize=(11,8.5))

ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.0))

cs = ax.pcolormesh(model_interp['lon'],model_interp['lat'],
                   model_interp['sst'],cmap='coolwarm',transform=ccrs.PlateCarree())
ax.coastlines()
plt.title('Interpolated')

In [ ]:
fig = plt.figure(figsize=(11,8.5))

ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.0))

cs = ax.pcolormesh(ds_model_mean['lon'],ds_model_mean['lat'],
                   ds_model_mean['sst'],cmap='coolwarm',transform=ccrs.PlateCarree())
ax.coastlines()
plt.title('Original')

In [ ]:
diff = (model_interp-ds_obs_mean).where(ds_mask['mask']==1)
diff

In [ ]:
from matplotlib.colors import CenteredNorm
fig = plt.figure(figsize=(11,7))

plt.pcolormesh(diff.lon,diff.lat,diff['sst'], cmap='coolwarm', norm=CenteredNorm(0))
plt.colorbar(label='˚C')
plt.title('Model minus OBS')